## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 64
num_epoch  = 300

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [150, 225]

pretrained_model       = './cifar100_densenet100_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


## Define densenet 100

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarDensenetConfig(k = 12, num_layers = 100, cifar = 100)
model = model_gen.GetCifarDensenet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

## Training (back-propagation)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 3.768283, Test Acc: 18.26 %
2 epoch end, loss: 2.865367, Test Acc: 32.97 %
3 epoch end, loss: 2.340488, Test Acc: 40.26 %
4 epoch end, loss: 2.043674, Test Acc: 42.03 %
5 epoch end, loss: 1.848320, Test Acc: 46.39 %
6 epoch end, loss: 1.702069, Test Acc: 50.71 %
7 epoch end, loss: 1.601929, Test Acc: 50.75 %
8 epoch end, loss: 1.514263, Test Acc: 53.10 %
9 epoch end, loss: 1.444686, Test Acc: 54.76 %
10 epoch end, loss: 1.383891, Test Acc: 56.10 %
11 epoch end, loss: 1.338831, Test Acc: 55.00 %
12 epoch end, loss: 1.286591, Test Acc: 55.96 %
13 epoch end, loss: 1.252058, Test Acc: 56.03 %
14 epoch end, loss: 1.222591, Test Acc: 58.50 %
15 epoch end, loss: 1.185899, Test Acc: 59.03 %
16 epoch end, loss: 1.160858, Test Acc: 56.64 %
17 epoch end, loss: 1.134885, Test Acc: 61.83 %
18 epoch end, loss: 1.112311, Test Acc: 58.22 %
19 epoch end, loss: 1.089871, Test Acc: 58.13 %
20 epoch end, loss: 1.073346, Test Acc: 59.60 %
21 epoch end, loss: 1.055414, Test Acc: 61.19 %
2

172 epoch end, loss: 0.120345, Test Acc: 74.29 %
173 epoch end, loss: 0.127791, Test Acc: 73.52 %
174 epoch end, loss: 0.132444, Test Acc: 74.11 %
175 epoch end, loss: 0.131736, Test Acc: 73.44 %
176 epoch end, loss: 0.130261, Test Acc: 73.75 %
177 epoch end, loss: 0.132178, Test Acc: 73.86 %
178 epoch end, loss: 0.137311, Test Acc: 72.98 %
179 epoch end, loss: 0.137478, Test Acc: 72.90 %
180 epoch end, loss: 0.145128, Test Acc: 73.13 %
181 epoch end, loss: 0.149313, Test Acc: 73.20 %
182 epoch end, loss: 0.149911, Test Acc: 72.33 %
183 epoch end, loss: 0.153381, Test Acc: 73.40 %
184 epoch end, loss: 0.155578, Test Acc: 72.39 %
185 epoch end, loss: 0.169941, Test Acc: 72.26 %
186 epoch end, loss: 0.161928, Test Acc: 73.26 %
187 epoch end, loss: 0.161438, Test Acc: 72.74 %
188 epoch end, loss: 0.154683, Test Acc: 72.83 %
189 epoch end, loss: 0.168108, Test Acc: 72.32 %
190 epoch end, loss: 0.163830, Test Acc: 71.61 %
191 epoch end, loss: 0.173155, Test Acc: 72.92 %
192 epoch end, loss:

## Fine-tuning (KD + Cross-entropy)

In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 76.46 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)

In [11]:
net.blocks

[[Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  'ConvBlock'],
 [[BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2